In [7]:
import os
os.environ["PATH"] = os.environ.get("PATH", "") + ":/opt/ffmpeg-btbn/ffmpeg-*-linux64-gpl/bin"
os.environ["LIBVMAF_MODEL_PATH"] = "/usr/local/share/model"

import subprocess
from IPython.display import display

print(subprocess.check_output(["ffmpeg","-version"], text=True).splitlines()[0])
print("libvmaf" in subprocess.check_output(["ffmpeg","-hide_banner","-filters"], text=True))

ffmpeg version N-121432-g6b961f5963-20251014 Copyright (c) 2000-2025 the FFmpeg developers
True


In [ ]:
# One-cell Jupyter notebook: per-part HEVC compression with 6s samples + FAST VMAF only
# Phase A (samples): choose guidance by S on two 6s samples per part (fast VMAF)
# Phase B (full part): ALWAYS start at CRF=34, then step down (32→30→28→26) ONLY IF (VMAF<80 AND S<0.40)
# Keep same resolution & extension. Final metrics computed with FAST VMAF on the combined file.

# ====== PATHS ======
INPUT_DIR  = "/home/indranil/video_process/testbench1"
OUTPUT_DIR = "/home/indranil/video_process/testbench1_output"

import os, json, pathlib, subprocess, tempfile, shutil, math
from typing import List, Dict, Tuple, Optional

os.makedirs(OUTPUT_DIR, exist_ok=True)

# ====== POLICY / KNOBS ======
PART_MAX_SEC = 30.0                          # part length target (no short tail)
CRF_ORDER    = [34, 32, 30, 28, 26]          # Phase B always starts from 34

TARGET_VMAF_MIN          = 80.0              # acceptance rule
TARGET_S_ACCEPT_MIN      = 0.40              # acceptance rule (S >= 0.40)
TARGET_C_MAX_FINAL       = 0.70              # for final reporting (not a hard stop in flow)

# VMAF knobs (FAST everywhere)
VMAF_SUBSAMPLE = 5                            # every 5th frame
VMAF_HALFRES   = False                        # full-res frames; subsampling handles speed
VMAF_THREADS   = 1

# Sampling windows (each 6s)
SAMPLE_LEN = 6.0

# FFmpeg threading
FFMPEG_THREADS = "1"

# Encoders & defaults
X265_PRESET   = "medium"                      # final encodes and sample encodes use same preset for stability
X264_PRESET   = "medium"
X265_PROFILE  = "main10"                      # or "main"
X265_LEVEL    = "5.1"
X265_PARAMS   = {
    "aq-mode": 2, "aq-strength": 1.0,
    "psy-rd": 1.0, "psy-rdoq": 1.0,
    "rd": 4,
    "pools": 1, "frame-threads": 1
}
AUDIO_CODECS = {
    ".mp4": ("aac","128k",48000),
    ".mov": ("aac","192k",48000),
    ".mkv": ("copy",None,None),
    ".avi": ("libmp3lame","192k",48000),
    ".mpg": ("copy",None,None)
}
ALLOWED_EXTS = {".mp4",".mkv",".mov",".avi",".mpg"}

# ====== UTILS ======
def run(cmd: List[str]) -> Tuple[int,str,str]:
    p = subprocess.run(cmd, capture_output=True, text=True)
    return p.returncode, p.stdout, p.stderr

def ffprobe_json(path: str) -> dict:
    code, out, err = run(["ffprobe","-v","error","-print_format","json","-show_streams","-show_format", path])
    if code != 0:
        raise RuntimeError(f"ffprobe failed: {err}")
    return json.loads(out)

def duration_sec(path: str) -> float:
    j = ffprobe_json(path)
    return float(j["format"].get("duration", 0.0))

def file_size(path: str) -> int:
    return os.path.getsize(path)

def ext_of(path: str) -> str:
    return pathlib.Path(path).suffix.lower()

def out_with_tag(src: str, out_dir: str, tag: str) -> str:
    p = pathlib.Path(src)
    return str(pathlib.Path(out_dir) / f"{p.stem}_{tag}{p.suffix.lower()}")

def out_with_tag_ext(src_like: str, out_dir: str, tag: str, target_ext: str) -> str:
    stem = pathlib.Path(src_like).stem
    return str(pathlib.Path(out_dir) / f"{stem}_{tag}{target_ext.lower()}")

def s_metric(C: float, vmaf: float) -> float:
    # S = 0.8*(1 - C^1.5) + 0.2*((VMAF - 80) / 20)
    return 0.8*(1 - (C**1.5)) + 0.2*((vmaf - 80.0)/20.0)

def list_input_videos(input_dir: str) -> List[str]:
    vids = []
    for name in sorted(os.listdir(input_dir)):
        p = str(pathlib.Path(input_dir) / name)
        if os.path.isfile(p) and ext_of(p) in ALLOWED_EXTS:
            vids.append(p)
    return vids

def check_ffmpeg_stack():
    if run(["ffmpeg","-version"])[0] != 0:
        raise RuntimeError("ffmpeg not found")
    if "libvmaf" not in run(["ffmpeg","-hide_banner","-filters"])[1]:
        raise RuntimeError("ffmpeg built without libvmaf")
    encs = run(["ffmpeg","-hide_banner","-encoders"])[1]
    if "libx265" not in encs:
        print("WARN: libx265 missing (HEVC encodes will fail)")
    if "libx264" not in encs:
        print("WARN: libx264 missing (AVI fallback will fail)")

# ====== FORMAT-AWARE FILTERS ======
def is_hdr_like(path: str) -> bool:
    try:
        j = ffprobe_json(path)
        v = next(s for s in j["streams"] if s.get("codec_type")=="video")
        tr = (v.get("color_transfer") or "").lower()
        return ("2084" in tr) or ("hlg" in tr) or ("b67" in tr)
    except Exception:
        return False

def is_mpg_sd(path: str) -> bool:
    try:
        j = ffprobe_json(path)
        v = next(s for s in j["streams"] if s.get("codec_type")=="video")
        return int(v.get("width", 1920)) <= 1024
    except Exception:
        return False

def x265_params_string(d: Dict) -> str:
    return ":".join(f"{k}={v}" for k,v in d.items())

def build_encode_cmd(src: str, dst: str, crf: int, *, vf_extra: Optional[List[str]]=None) -> List[str]:
    """
    Encodes (samples and full parts). Same resolution; color/deint normalization for MPG/HDR.
    hvc1 tag only for MP4/MOV.
    """
    ext = ext_of(dst)
    vf = []
    if ext == ".mpg":
        matrix = "bt601" if is_mpg_sd(src) else "bt709"
        vf += ["bwdif=mode=send_frame:parity=auto:deint=all",
               f"zscale=matrix={matrix}:transfer={matrix}:primaries={matrix}",
               "format=yuv420p","setsar=1"]
    elif is_hdr_like(src):
        vf += ["zscale=t=linear","tonemap=hable",
               "zscale=matrix=bt709:transfer=bt709:primaries=bt709",
               "format=yuv420p","setsar=1"]
    else:
        vf += ["format=yuv420p","setsar=1"]

    if vf_extra:
        vf += vf_extra

    vf_arg = ",".join(vf) if vf else None
    base = ["ffmpeg","-hide_banner","-loglevel","error","-nostdin","-threads",FFMPEG_THREADS, "-y", "-i", src]
    if vf_arg:
        base += ["-vf", vf_arg]

    if ext in (".mp4",".mkv",".mov",".mpg"):
        v = ["-c:v","libx265","-preset", X265_PRESET, "-crf", str(crf),
             "-profile:v", X265_PROFILE, "-level", X265_LEVEL,
             "-x265-params", x265_params_string(X265_PARAMS),
             "-pix_fmt","yuv420p"]
        if ext in (".mp4",".mov"):
            v += ["-tag:v","hvc1"]
        ac, abr, ar = AUDIO_CODECS.get(ext, ("copy",None,None))
        a = ["-c:a","copy"] if ac=="copy" else ["-c:a",ac] + (["-b:a",abr] if abr else []) + (["-ar",str(ar)] if ar else [])
        if ext in (".mp4",".mov"):
            a += ["-movflags","+faststart"]
        return base + v + a + [dst]
    elif ext == ".avi":
        v = ["-c:v","libx264","-preset", X264_PRESET, "-crf", str(crf), "-pix_fmt","yuv420p"]
        ac, abr, ar = AUDIO_CODECS.get(ext, ("libmp3lame","192k",48000))
        a = ["-c:a", ac, "-b:a", abr, "-ar", str(ar)]
        return base + v + a + [dst]
    else:
        raise ValueError(f"Unsupported extension: {ext}")

def build_vmaf_filter(src_for_norm: str, n_subsample: Optional[int], half_res: bool, vmaf_threads: int) -> str:
    ext = ext_of(src_for_norm)
    def mpg_chain():
        matrix = "bt601" if is_mpg_sd(src_for_norm) else "bt709"
        return ["zscale=matrix={}:transfer={}:primaries={}".format(matrix, matrix, matrix),
                "format=yuv420p","setsar=1"]
    def hdr_chain():
        return ["zscale=t=linear","tonemap=hable",
                "zscale=matrix=bt709:transfer=bt709:primaries=bt709",
                "format=yuv420p","setsar=1"]

    if ext == ".mpg":
        ref = ["setpts=PTS-STARTPTS","bwdif=mode=send_frame:parity=auto:deint=all", *mpg_chain()]
        dist= ["setpts=PTS-STARTPTS","bwdif=mode=send_frame:parity=auto:deint=all", *mpg_chain()]
    elif is_hdr_like(src_for_norm):
        ref = ["setpts=PTS-STARTPTS", *hdr_chain()]
        dist= ["setpts=PTS-STARTPTS", *hdr_chain()]
    else:
        ref = ["setpts=PTS-STARTPTS","format=yuv420p","setsar=1"]
        dist= ["setpts=PTS-STARTPTS","format=yuv420p","setsar=1"]

    if half_res:
        ref.insert(-2, "scale=iw/2:ih/2:flags=bicubic")
        dist.insert(-2, "scale=iw/2:ih/2:flags=bicubic")

    ref_s  = ",".join(ref)  + "[ref]"
    dist_s = ",".join(dist) + "[dist]"
    opts   = [f"n_threads={vmaf_threads}", "log_fmt=json", "log_path='${LOGP}'"]
    if n_subsample and n_subsample > 1:
        opts.append(f"n_subsample={n_subsample}")
    return f"[0:v]{ref_s};[1:v]{dist_s};[dist][ref]libvmaf=" + ":".join(opts)

def compute_vmaf_fast(src: str, dist: str, *, subsample: int = VMAF_SUBSAMPLE, halfres: bool = VMAF_HALFRES) -> float:
    with tempfile.TemporaryDirectory() as td:
        logp = os.path.join(td, "vmaf.json")
        filt = build_vmaf_filter(src, subsample, halfres, VMAF_THREADS).replace("${LOGP}", logp)
        code, out, err = run(["ffmpeg","-hide_banner","-loglevel","error","-nostdin",
                              "-i", src, "-i", dist, "-lavfi", filt, "-f","null","-"])
        if code != 0:
            raise RuntimeError(f"VMAF failed: {err}")
        data = json.load(open(logp))
        try:
            return float(data["pooled_metrics"]["vmaf"]["mean"])
        except Exception:
            frames = data.get("frames", [])
            vals = [fr["metrics"]["vmaf"] for fr in frames if "metrics" in fr and "vmaf" in fr["metrics"]]
            if not vals:
                raise RuntimeError("VMAF JSON missing values")
            return sum(vals)/len(vals)

# ====== SPLIT & EXTRACT ======
def split_into_parts_no_short_tail(total_sec: float, max_len_sec: float = PART_MAX_SEC) -> List[Tuple[float,float]]:
    """
    Split into chunks of <= max_len_sec, but if the remainder is < max_len_sec,
    merge it into the previous chunk (unless whole video < max_len_sec).
    """
    T = max(0.0, float(total_sec))
    if T <= 0:
        return [(0.0, 0.0)]
    if T <= max_len_sec:
        return [(0.0, T)]
    n_full = int(T // max_len_sec)
    rem    = T - n_full*max_len_sec
    parts = []
    if rem == 0:
        for i in range(n_full):
            parts.append((i*max_len_sec, max_len_sec))
    else:
        if n_full == 0:
            parts.append((0.0, T))
        else:
            for i in range(n_full-1):
                parts.append((i*max_len_sec, max_len_sec))
            last_start = (n_full-1)*max_len_sec
            last_dur   = max_len_sec + rem
            parts.append((last_start, last_dur))
    return parts

def extract_part_to_mkv(src: str, start: float, dur: float) -> str:
    tmpdir = tempfile.mkdtemp()
    dst = os.path.join(tmpdir, f"part_{int(start)}_{int(start+dur)}.mkv")
    code, _, err = run([
        "ffmpeg","-hide_banner","-loglevel","error","-nostdin",
        "-ss", str(start), "-t", str(dur), "-i", src,
        "-map","0","-c","copy",
        "-fflags","+genpts","-copyts","-avoid_negative_ts","make_zero",
        "-f","matroska", dst
    ])
    if code != 0 or not os.path.exists(dst) or os.path.getsize(dst)==0:
        shutil.rmtree(tmpdir, ignore_errors=True)
        raise RuntimeError(f"extract_part to MKV failed: {err}")
    return dst

# ====== SAMPLING WINDOWS ======
def pick_sample_windows(part_duration: float) -> List[Tuple[float,float]]:
    d = part_duration
    if d <= 0:
        return []
    if d <= SAMPLE_LEN:
        return [(max(0.0, (d - SAMPLE_LEN)/2.0), min(SAMPLE_LEN, d))]
    if d <= 2*SAMPLE_LEN:
        return [(0.0, SAMPLE_LEN), (max(0.0, d - SAMPLE_LEN), SAMPLE_LEN)]
    s1 = max(0.0, min(d - SAMPLE_LEN, 0.25*d))
    s2 = max(0.0, min(d - SAMPLE_LEN, 0.75*d))
    if abs(s2 - s1) < 1e-3:
        s2 = min(d - SAMPLE_LEN, s1 + SAMPLE_LEN + 0.01)
    return [(s1, SAMPLE_LEN), (s2, SAMPLE_LEN)]

def extract_sample_mkv(part_src_mkv: str, start: float, dur: float) -> str:
    tmpdir = tempfile.mkdtemp()
    dst = os.path.join(tmpdir, f"sample_{int(start)}_{int(start+dur)}.mkv")
    code, _, err = run([
        "ffmpeg","-hide_banner","-loglevel","error","-nostdin",
        "-ss", str(start), "-t", str(dur), "-i", part_src_mkv,
        "-map","0","-c","copy",
        "-fflags","+genpts","-copyts","-avoid_negative_ts","make_zero",
        "-f","matroska", dst
    ])
    if code != 0 or not os.path.exists(dst) or os.path.getsize(dst)==0:
        shutil.rmtree(tmpdir, ignore_errors=True)
        raise RuntimeError(f"extract_sample to MKV failed: {err}")
    return dst

# ====== SAMPLE EVAL (per CRF, fast VMAF) ======
def eval_crf_on_samples(part_src_mkv: str, crf: int, original_ext: str) -> Dict:
    """
    For the given part and CRF:
      - pick 2×6s windows (or 1 if short),
      - encode each sample (same extension as final container, short segment),
      - compute FAST VMAF and S for each, then average.
    Returns {crf, avg_vmaf, avg_C, avg_S}.
    """
    part_dur = duration_sec(part_src_mkv)
    windows = pick_sample_windows(part_dur)
    if not windows:
        raise RuntimeError("No sampling windows")

    S_vals, V_vals, C_vals = [], [], []
    sample_refs, sample_outs = [], []
    try:
        for (ss, dd) in windows:
            ref = extract_sample_mkv(part_src_mkv, ss, dd)
            out = out_with_tag_ext(ref, tempfile.mkdtemp(), f"crf{crf}_sample_enc", original_ext)
            code, _, err = run(build_encode_cmd(ref, out, crf))
            if code != 0 or not os.path.exists(out):
                try: shutil.rmtree(os.path.dirname(out), ignore_errors=True)
                except: pass
                try: shutil.rmtree(os.path.dirname(ref), ignore_errors=True)
                except: pass
                continue
            try:
                V = compute_vmaf_fast(ref, out, subsample=VMAF_SUBSAMPLE, halfres=False)
                C = file_size(out)/file_size(ref)
                S = s_metric(C, V)
                S_vals.append(S); V_vals.append(V); C_vals.append(C)
                sample_refs.append(ref); sample_outs.append(out)
            except Exception:
                try: os.remove(out)
                except: pass
                try: shutil.rmtree(os.path.dirname(out), ignore_errors=True)
                except: pass
                try: shutil.rmtree(os.path.dirname(ref), ignore_errors=True)
                except: pass
                continue
        if not S_vals:
            raise RuntimeError("All sample encodes failed for this CRF")

        return {
            "crf": crf,
            "avg_S": sum(S_vals)/len(S_vals),
            "avg_vmaf": sum(V_vals)/len(V_vals),
            "avg_C": sum(C_vals)/len(C_vals),
            "n_samples": len(S_vals)
        }
    finally:
        for p in sample_outs:
            try:
                os.remove(p)
                shutil.rmtree(os.path.dirname(p), ignore_errors=True)
            except: pass
        for p in sample_refs:
            try: shutil.rmtree(os.path.dirname(p), ignore_errors=True)
            except: pass

# ====== PHASE A + PHASE B PER PART ======
def process_part_seq(part_src_mkv: str, out_dir: str, original_ext: str) -> Dict:
    """
    Phase A (Samples): score CRFs on 2×6s windows using FAST VMAF; this is for guidance/reporting.
    Phase B (Full Part): ALWAYS start from CRF=34, then step down if (VMAF<80 AND S<0.40). Keep best S seen.
    If all encodes fail, return the best S record obtained during Phase B (or remux if nothing succeeded).
    """
    part_bytes = file_size(part_src_mkv)

    # ---- Phase A: sample-based sweep (guidance) ----
    print("[SAMPLE] Begin")
    sample_scores = []
    for crf in CRF_ORDER:
        try:
            r = eval_crf_on_samples(part_src_mkv, crf, original_ext)
            sample_scores.append(r)
            print(f"  CRF{crf}: avg_S={r['avg_S']:.3f} avg_VMAF={r['avg_vmaf']:.2f} avg_C={r['avg_C']:.3f} samples={r['n_samples']}")
        except Exception as e:
            print(f"  CRF{crf} sample eval failed: {e}")
    if sample_scores:
        # Display best-by-sample (not used to start order anymore, but useful log)
        best_sample = sorted(sample_scores, key=lambda r: (r["avg_S"], r["avg_vmaf"], -r["avg_C"]), reverse=True)[0]
        print(f"[SAMPLE] Best by samples: CRF{best_sample['crf']} (avg_S={best_sample['avg_S']:.3f})")
    else:
        print("[SAMPLE] No sample results; proceeding to full part checks.")

    # ---- Phase B: full part encodes, start at CRF=34 then step down only if (VMAF<80 AND S<0.40) ----
    print("[PART] Full-part phase (CRF 34→26 as needed)")
    best = None  # (S, record)
    any_success = False

    for crf in CRF_ORDER:  # fixed order: 34,32,30,28,26
        part_out = out_with_tag_ext(part_src_mkv, out_dir, f"hevc_part_crf{crf}", original_ext)
        code, _, err = run(build_encode_cmd(part_src_mkv, part_out, crf))
        if code != 0 or not os.path.exists(part_out):
            if os.path.exists(part_out):
                try: os.remove(part_out)
                except: pass
            print(f"  CRF{crf}: encode failed, skipping.")
            continue

        any_success = True
        try:
            V = compute_vmaf_fast(part_src_mkv, part_out)
            C = file_size(part_out)/part_bytes
            S = s_metric(C, V)
            rec = {"part_src": part_src_mkv, "output": part_out, "crf": crf, "VMAF": V, "C": C, "S": S,
                   "met_targets": (V >= TARGET_VMAF_MIN)}
            print(f"  CRF{crf}: VMAF={V:.2f} C={C:.3f} S={S:.3f}")
            if (best is None) or (S > best[0]):
                best = (S, rec)

            # Acceptance: if NOT (VMAF<80 AND S<0.40) → accept now
            if not (V < TARGET_VMAF_MIN and S < TARGET_S_ACCEPT_MIN):
                print(f"  CRF{crf}: accepted (VMAF>=80 OR S>=0.40).")
                return rec
            else:
                print(f"  CRF{crf}: insufficient (VMAF<80 AND S<0.40). Trying lower CRF...")
                continue

        except Exception as e:
            print(f"  CRF{crf}: VMAF failed: {e}")
            try: os.remove(part_out)
            except: pass
            continue

    # If we reached here and had successes, return best S found
    if any_success and best is not None:
        print(f"[PART] No accept by rule; keeping best S={best[0]:.3f} at CRF{best[1]['crf']}.")
        return best[1]

    # If absolutely everything failed, remux original part to let pipeline finish
    print("[PART] All encodes failed; remux original part.")
    remux_out = out_with_tag_ext(part_src_mkv, out_dir, "remux_after_all_fail", original_ext)
    code_r, _, err_r = run(["ffmpeg","-hide_banner","-loglevel","error","-nostdin","-i", part_src_mkv, "-c","copy", remux_out])
    if code_r == 0 and os.path.exists(remux_out):
        try: V = compute_vmaf_fast(part_src_mkv, remux_out)
        except Exception: V = TARGET_VMAF_MIN
        C = file_size(remux_out)/part_bytes
        S = s_metric(C, V)
        return {"part_src": part_src_mkv, "output": remux_out, "crf": None, "VMAF": V, "C": C, "S": S,
                "met_targets": (V>=TARGET_VMAF_MIN), "note": "remux_after_all_fail"}
    else:
        raise RuntimeError(f"Part remux failed: {err_r}")

# ====== CONCAT & FINAL METRICS (FAST) ======
def concat_parts(outputs_in_order: List[str], final_path: str) -> None:
    with tempfile.NamedTemporaryFile(delete=False, mode="w", suffix=".txt") as tf:
        listp = tf.name
        for p in outputs_in_order:
            tf.write(f"file '{p}'\n")
    code, _, err = run(["ffmpeg","-hide_banner","-loglevel","error","-nostdin",
                        "-f","concat","-safe","0","-i", listp,"-c","copy", final_path])
    os.remove(listp)
    if code != 0 or not os.path.exists(final_path):
        raise RuntimeError(f"Concat failed: {err}")

def final_metrics_fast(src: str, dst: str) -> Dict:
    V = compute_vmaf_fast(src, dst, subsample=VMAF_SUBSAMPLE, halfres=False)
    C = file_size(dst)/file_size(src)
    S = s_metric(C, V)
    return {"VMAF": V, "C": C, "S": S}

# ====== DRIVER: one video by index ======
def run_one_video_by_index(index: int) -> Dict:
    check_ffmpeg_stack()

    vids = list_input_videos(INPUT_DIR)
    if not vids:
        raise RuntimeError(f"No videos found in {INPUT_DIR}")
    print("Indexed videos:")
    for i,v in enumerate(vids):
        print(f"[{i}] {v}")
    if index < 0 or index >= len(vids):
        raise IndexError(f"INDEX {index} out of range (0..{len(vids)-1})")

    src = vids[index]
    print(f"\nSelected: [{index}] {src}")
    T = duration_sec(src)
    print(f"Duration: {T:.2f}s")

    # Split parts (no tail <30s)
    parts = split_into_parts_no_short_tail(T, PART_MAX_SEC)
    print(f"Parts: {len(parts)} -> {parts}")

    # Extract MKV parts
    part_files = [extract_part_to_mkv(src, s, d) for (s,d) in parts]
    print("Part Files Determined")

    # Process sequentially
    orig_ext = ext_of(src)
    part_results = []
    try:
        for i, p in enumerate(part_files):
            print(f"\n=== PART {i} ===")
            r = process_part_seq(p, OUTPUT_DIR, orig_ext)
            print(f"[PART {i}] Result: CRF={r['crf']} VMAF={r['VMAF']:.2f} C={r['C']:.3f} S={r['S']:.3f}")
            part_results.append(r)
    finally:
        # cleanup extracted part sources
        for p in part_files:
            try: shutil.rmtree(os.path.dirname(p), ignore_errors=True)
            except: pass
    print("Processed Parts Sequentially complete")

    # Concat to final (same extension as source)
    final_out = out_with_tag(src, OUTPUT_DIR, "hevc_final_concat")
    concat_parts([r["output"] for r in part_results], final_out)

    # Final whole-video fast VMAF
    fin = final_metrics_fast(src, final_out)
    ok = (fin["VMAF"] >= TARGET_VMAF_MIN) and (fin["C"] <= TARGET_C_MAX_FINAL)
    print("Final Metrics complete")

    print("\n=== Per-part results ===")
    for i, r in enumerate(part_results):
        print(f"Part {i}: CRF={r['crf']}, C={r['C']:.4f}, VMAF={r['VMAF']:.2f}, S={r['S']:.3f}, ok={r['met_targets']}")

    print("\n=== Final (whole video) ===")
    print({"output": final_out, "C": round(fin["C"],4), "VMAF": round(fin["VMAF"],2), "S": round(fin["S"],3), "ok": ok})

    return {"source": src, "final_output": final_out, "final_metrics": fin, "per_part": part_results, "ok": ok}

# -------- Example: set an index and run ----------


In [ ]:
INDEX = 0
result = run_one_video_by_index(INDEX)